## 국민청원 크롤링하기

In [1]:
from bs4 import BeautifulSoup 
from urllib.request import urlopen

import pandas as pd
import re
import time

from datetime import datetime, timedelta
from selenium import webdriver

#### 국민청원 분야별 게시판 주소
- 외교/통일/국방 : 36
- 일자리 : 37
- 인권/성평등 : 49

In [2]:
urls = []
url = input("크롤링할 청원 분야를 입력하세요.\n- 일교/통일/국방 : 36 \n- 일자리 : 37 \n- 인권/성평등 : 49 \n")
for x in range(1, 2000):
    urls.append('https://www1.president.go.kr/petitions/category?c=%s&only=2&page=%d&order=1' % (url, x))

크롤링할 청원 분야를 입력하세요.
- 일교/통일/국방 : 36 
- 일자리 : 37 
- 인권/성평등 : 49 
37


In [3]:
len(urls)

1999

In [4]:
urls[0]

'https://www1.president.go.kr/petitions/category?c=37&only=2&page=1&order=1'

### 하나의 페이지 테스트

In [5]:
driver_path = "driver/chromedriver.exe"
driver = webdriver.Chrome(executable_path=driver_path)
driver.get(urls[0])
data = BeautifulSoup(driver.page_source, "lxml")

In [6]:
data

<!DOCTYPE html>
<html class=" js csstransitions" lang="ko" style="height: auto;" xmlns="http://www.w3.org/1999/xhtml"><head>
<meta charset="utf-8"/>
<meta content="IE=edge, chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width,initial-scale=1.0,minimum-scale=1.0,maximum-scale=1.0" name="viewport"/>
<meta content="#082e59" name="theme-color"/>
<meta content="나라를 나라답게, 국민과 함께 갑니다." name="description"/>
<meta content="청와대, 대통령비서실, 대통령, 한국, 대한민국, 대한민국 정부, president.go.kr, 靑瓦臺, 19대, 문재인" name="keywords"/>
<meta content="청와대 (CHEONGWADAE)" name="author"/>
<meta content="telephone=no" name="format-detection"/>
<meta content="website" property="og:type"/>
<meta content="분야별 국민청원 및 제안 목록 페이지 1" property="og:title"/>
<meta content="http://www.president.go.kr/" property="og:url"/>
<meta content="분야별 국민청원 및 제안 목록 페이지 1" property="og:site_name"/>
<meta content="나라를 나라답게, 국민과 함께 갑니다." property="og:description"/>
<meta content="http://president.go.kr/css/favicon_400x210.png" prop

In [7]:
data_list = data.find('ul', 'petition_list').find_all('li')

In [8]:
data_list[0]

<li style="padding:20px 0;"><div class="bl_wrap"><div class="bl_no"><span class="sound_only">번호 </span>17619</div><div class="bl_category ccategory cs"><span class="sound_only">분류 </span>일자리</div><div class="bl_subject"><a class="cb relpy_w" href="/petitions/409768"><span class="sound_only">제목 </span>카카오톡 카플택시</a></div><div class="bl_name"><span class="sound_only">청원인 </span>naver - ***</div><div class="bl_date light"><span class="sound_only">청원기간 </span>18.10.17 ~ 18.11.16</div><div class="bl_agree cs"><span class="sound_only">참여인원 </span>40명</div></div></li>

In [9]:
data_list[-1]

<li style="padding:20px 0;"><div class="bl_wrap"><div class="bl_no"><span class="sound_only">번호 </span>17610</div><div class="bl_category ccategory cs"><span class="sound_only">분류 </span>일자리</div><div class="bl_subject"><a class="cb relpy_w" href="/petitions/409571"><span class="sound_only">제목 </span>문제인씨   도대체   당신의    정체가   뭐요?</a></div><div class="bl_name"><span class="sound_only">청원인 </span>facebook - ***</div><div class="bl_date light"><span class="sound_only">청원기간 </span>18.10.17 ~ 18.11.16</div><div class="bl_agree cs"><span class="sound_only">참여인원 </span>17명</div></div></li>

#### 청원 제목 뽑기

In [10]:
data_list[0].find('a', 'cb relpy_w')

<a class="cb relpy_w" href="/petitions/409768"><span class="sound_only">제목 </span>카카오톡 카플택시</a>

In [11]:
data_list[-1].find('a', 'cb relpy_w')

<a class="cb relpy_w" href="/petitions/409571"><span class="sound_only">제목 </span>문제인씨   도대체   당신의    정체가   뭐요?</a>

In [12]:
data_list[0].find('a', 'cb relpy_w').get_text()

'제목 카카오톡 카플택시'

In [13]:
data_list[0].find('a', 'cb relpy_w').get_text()[3:]

'카카오톡 카플택시'

#### 청원 날짜 뽑기

In [14]:
data_list[0].find('div', 'bl_date light')

<div class="bl_date light"><span class="sound_only">청원기간 </span>18.10.17 ~ 18.11.16</div>

In [15]:
data_list[-1].find('div', 'bl_date light')

<div class="bl_date light"><span class="sound_only">청원기간 </span>18.10.17 ~ 18.11.16</div>

In [16]:
data_list[0].find('div', 'bl_date light').get_text()

'청원기간 18.10.17 ~ 18.11.16'

In [17]:
data_list[0].find('div', 'bl_date light').get_text()[5:]

'18.10.17 ~ 18.11.16'

#### 청원 참여인원 뽑기

In [18]:
data_list[0].find('div', 'bl_agree cs')

<div class="bl_agree cs"><span class="sound_only">참여인원 </span>40명</div>

In [19]:
data_list[1].find('div', 'bl_agree cs')

<div class="bl_agree cs"><span class="sound_only">참여인원 </span>9명</div>

In [20]:
data_list[0].find('div', 'bl_agree cs').get_text()

'참여인원 40명'

In [21]:
data_list[0].find('div', 'bl_agree cs').get_text()[5:13]

'40명'

### 데이터 수집 및 확인

In [22]:
title = []
date = []
count = []
page = 0

data1 = []

start_flag = False
end_flag = False

end_date = input("시작 날짜를 입력하세요.  ex) 181007 \n")
start_date = input("마지막 날짜를 입력하세요.  ex) 181115 \n")
end_date = end_date[:2] + '.' + end_date[2:4] + '.' + end_date[4:]
start_date = start_date[:2] + '.' + start_date[2:4] + '.' + start_date[4:]

print("=" * 50)
print("[%s ~ %s] 국민청원 데이터를 수집합니다.\n" % (end_date, start_date))

start = datetime.now()

# 가상 드라이버 시작
driver_path = "driver/chromedriver.exe"
driver = webdriver.Chrome(executable_path=driver_path)
time.sleep(1)

for url in urls:
    time.sleep(3)
    # url 이동 및 html 소스 수집
    driver.get(url)
    time.sleep(2)
    data = BeautifulSoup(driver.page_source, "lxml")
    time.sleep(2)
    data1.append(data)
    
#     print(data)
    
    # 청원 데이터 찾기
    data_list = data.find('ul', 'petition_list').find_all('li')
#     print(data_list)
    
    # 데이터 수집 체크
    page += 1
#     print(page)
    
    if start_flag:
        print("%d 페이지 수집중입니다.... ing" %page, end='')
    
    for x in range(len(data_list)):
        
        # 기사 끝날 날짜 체크 (end_flag 변경)
        if data_list[x].find('div', 'bl_date light').get_text()[-8:] in end_date:
            temp_date = data_list[x].find('div', 'bl_date light').get_text()[-8:]
            end_flag = True
               

        # 기사 끝날 날짜 체크(반복문 out)    
        if end_flag:
            if not(temp_date == data_list[x].find('div', 'bl_date light').get_text()[-8:]):
                break       
        
        
        if not(start_flag):            
            # 기사 시작 날짜 체크                                                                      
            if not(data_list[x].find('div', 'bl_date light').get_text()[-8:] in start_date):            
                continue
            print("%d 페이지 수집중입니다.... ing" %page, end='')
            start_flag = True        
  
        
        # 청원 제목만 가져오기
        title.append(data_list[x].find('a', 'cb relpy_w').get_text()[3:])
   
        # 청원 날짜만 가져오기
        date.append(data_list[x].find('div', 'bl_date light').get_text()[-8:])
        
        # 청원 참여인원 가져오기
        count.append(data_list[x].find('div', 'bl_agree cs').get_text()[5:])
    
    # 데이터 수집 시작 체크
    if not(start_flag):
        continue
        
    # 데이터수집 체크
    print("     [수집완료]")       
          
    
    # 기사 끝날 날짜 체크(반복문 out)  
    if end_flag:
        if not(temp_date == data_list[x].find('div', 'bl_date light').get_text()[-8:]):
            break 

        
end = datetime.now()
print("=" * 50)
print("데이터 수집을 완료했습니다.")
elapsed = end - start
print('총 계산 시간: ', end='');   print(elapsed)
print('총 수집 개수: %d개' %len(title))    

시작 날짜를 입력하세요.  ex) 181007 
180101
마지막 날짜를 입력하세요.  ex) 181115 
181115
[18.01.01 ~ 18.11.15] 국민청원 데이터를 수집합니다.

9 페이지 수집중입니다.... ing     [수집완료]
10 페이지 수집중입니다.... ing     [수집완료]
11 페이지 수집중입니다.... ing     [수집완료]
12 페이지 수집중입니다.... ing     [수집완료]
13 페이지 수집중입니다.... ing     [수집완료]
14 페이지 수집중입니다.... ing     [수집완료]
15 페이지 수집중입니다.... ing     [수집완료]
16 페이지 수집중입니다.... ing     [수집완료]
17 페이지 수집중입니다.... ing     [수집완료]
18 페이지 수집중입니다.... ing     [수집완료]
19 페이지 수집중입니다.... ing     [수집완료]
20 페이지 수집중입니다.... ing     [수집완료]
21 페이지 수집중입니다.... ing     [수집완료]
22 페이지 수집중입니다.... ing     [수집완료]
23 페이지 수집중입니다.... ing     [수집완료]
24 페이지 수집중입니다.... ing     [수집완료]
25 페이지 수집중입니다.... ing     [수집완료]
26 페이지 수집중입니다.... ing     [수집완료]
27 페이지 수집중입니다.... ing     [수집완료]
28 페이지 수집중입니다.... ing     [수집완료]
29 페이지 수집중입니다.... ing     [수집완료]
30 페이지 수집중입니다.... ing     [수집완료]
31 페이지 수집중입니다.... ing     [수집완료]
32 페이지 수집중입니다.... ing     [수집완료]
33 페이지 수집중입니다.... ing     [수집완료]
34 페이지 수집중입니다.... ing     [수집완료]
35 페이지 수집중입니다.... ing     [수집완료]
3

248 페이지 수집중입니다.... ing     [수집완료]
249 페이지 수집중입니다.... ing     [수집완료]
250 페이지 수집중입니다.... ing     [수집완료]
251 페이지 수집중입니다.... ing     [수집완료]
252 페이지 수집중입니다.... ing     [수집완료]
253 페이지 수집중입니다.... ing     [수집완료]
254 페이지 수집중입니다.... ing     [수집완료]
255 페이지 수집중입니다.... ing     [수집완료]
256 페이지 수집중입니다.... ing     [수집완료]
257 페이지 수집중입니다.... ing     [수집완료]
258 페이지 수집중입니다.... ing     [수집완료]
259 페이지 수집중입니다.... ing     [수집완료]
260 페이지 수집중입니다.... ing     [수집완료]
261 페이지 수집중입니다.... ing     [수집완료]
262 페이지 수집중입니다.... ing     [수집완료]
263 페이지 수집중입니다.... ing     [수집완료]
264 페이지 수집중입니다.... ing     [수집완료]
265 페이지 수집중입니다.... ing     [수집완료]
266 페이지 수집중입니다.... ing     [수집완료]
267 페이지 수집중입니다.... ing     [수집완료]
268 페이지 수집중입니다.... ing     [수집완료]
269 페이지 수집중입니다.... ing     [수집완료]
270 페이지 수집중입니다.... ing     [수집완료]
271 페이지 수집중입니다.... ing     [수집완료]
272 페이지 수집중입니다.... ing     [수집완료]
273 페이지 수집중입니다.... ing     [수집완료]
274 페이지 수집중입니다.... ing     [수집완료]
275 페이지 수집중입니다.... ing     [수집완료]
276 페이지 수집중입니다.... ing     [수집완료]
277 페이지 수집중입니다

489 페이지 수집중입니다.... ing     [수집완료]
490 페이지 수집중입니다.... ing     [수집완료]
491 페이지 수집중입니다.... ing     [수집완료]
492 페이지 수집중입니다.... ing     [수집완료]
493 페이지 수집중입니다.... ing     [수집완료]
494 페이지 수집중입니다.... ing     [수집완료]
495 페이지 수집중입니다.... ing     [수집완료]
496 페이지 수집중입니다.... ing     [수집완료]
497 페이지 수집중입니다.... ing     [수집완료]
498 페이지 수집중입니다.... ing     [수집완료]
499 페이지 수집중입니다.... ing     [수집완료]
500 페이지 수집중입니다.... ing     [수집완료]
501 페이지 수집중입니다.... ing     [수집완료]
502 페이지 수집중입니다.... ing     [수집완료]
503 페이지 수집중입니다.... ing     [수집완료]
504 페이지 수집중입니다.... ing     [수집완료]
505 페이지 수집중입니다.... ing     [수집완료]
506 페이지 수집중입니다.... ing     [수집완료]
507 페이지 수집중입니다.... ing     [수집완료]
508 페이지 수집중입니다.... ing     [수집완료]
509 페이지 수집중입니다.... ing     [수집완료]
510 페이지 수집중입니다.... ing     [수집완료]
511 페이지 수집중입니다.... ing     [수집완료]
512 페이지 수집중입니다.... ing     [수집완료]
513 페이지 수집중입니다.... ing     [수집완료]
514 페이지 수집중입니다.... ing     [수집완료]
515 페이지 수집중입니다.... ing     [수집완료]
516 페이지 수집중입니다.... ing     [수집완료]
517 페이지 수집중입니다.... ing     [수집완료]
518 페이지 수집중입니다

730 페이지 수집중입니다.... ing     [수집완료]
731 페이지 수집중입니다.... ing     [수집완료]
732 페이지 수집중입니다.... ing     [수집완료]
733 페이지 수집중입니다.... ing     [수집완료]
734 페이지 수집중입니다.... ing     [수집완료]
735 페이지 수집중입니다.... ing     [수집완료]
736 페이지 수집중입니다.... ing     [수집완료]
737 페이지 수집중입니다.... ing     [수집완료]
738 페이지 수집중입니다.... ing     [수집완료]
739 페이지 수집중입니다.... ing     [수집완료]
740 페이지 수집중입니다.... ing     [수집완료]
741 페이지 수집중입니다.... ing     [수집완료]
742 페이지 수집중입니다.... ing     [수집완료]
743 페이지 수집중입니다.... ing     [수집완료]
744 페이지 수집중입니다.... ing     [수집완료]
745 페이지 수집중입니다.... ing     [수집완료]
746 페이지 수집중입니다.... ing     [수집완료]
747 페이지 수집중입니다.... ing     [수집완료]
748 페이지 수집중입니다.... ing     [수집완료]
749 페이지 수집중입니다.... ing     [수집완료]
750 페이지 수집중입니다.... ing     [수집완료]
751 페이지 수집중입니다.... ing     [수집완료]
752 페이지 수집중입니다.... ing     [수집완료]
753 페이지 수집중입니다.... ing     [수집완료]
754 페이지 수집중입니다.... ing     [수집완료]
755 페이지 수집중입니다.... ing     [수집완료]
756 페이지 수집중입니다.... ing     [수집완료]
757 페이지 수집중입니다.... ing     [수집완료]
758 페이지 수집중입니다.... ing     [수집완료]
759 페이지 수집중입니다

971 페이지 수집중입니다.... ing     [수집완료]
972 페이지 수집중입니다.... ing     [수집완료]
973 페이지 수집중입니다.... ing     [수집완료]
974 페이지 수집중입니다.... ing     [수집완료]
975 페이지 수집중입니다.... ing     [수집완료]
976 페이지 수집중입니다.... ing     [수집완료]
977 페이지 수집중입니다.... ing     [수집완료]
978 페이지 수집중입니다.... ing     [수집완료]
979 페이지 수집중입니다.... ing     [수집완료]
980 페이지 수집중입니다.... ing     [수집완료]
981 페이지 수집중입니다.... ing     [수집완료]
982 페이지 수집중입니다.... ing     [수집완료]
983 페이지 수집중입니다.... ing     [수집완료]
984 페이지 수집중입니다.... ing     [수집완료]
985 페이지 수집중입니다.... ing     [수집완료]
986 페이지 수집중입니다.... ing     [수집완료]
987 페이지 수집중입니다.... ing     [수집완료]
988 페이지 수집중입니다.... ing     [수집완료]
989 페이지 수집중입니다.... ing     [수집완료]
990 페이지 수집중입니다.... ing     [수집완료]
991 페이지 수집중입니다.... ing     [수집완료]
992 페이지 수집중입니다.... ing     [수집완료]
993 페이지 수집중입니다.... ing     [수집완료]
994 페이지 수집중입니다.... ing     [수집완료]
995 페이지 수집중입니다.... ing     [수집완료]
996 페이지 수집중입니다.... ing     [수집완료]
997 페이지 수집중입니다.... ing     [수집완료]
998 페이지 수집중입니다.... ing     [수집완료]
999 페이지 수집중입니다.... ing     [수집완료]
1000 페이지 수집중입니

1206 페이지 수집중입니다.... ing     [수집완료]
1207 페이지 수집중입니다.... ing     [수집완료]
1208 페이지 수집중입니다.... ing     [수집완료]
1209 페이지 수집중입니다.... ing     [수집완료]
1210 페이지 수집중입니다.... ing     [수집완료]
1211 페이지 수집중입니다.... ing     [수집완료]
1212 페이지 수집중입니다.... ing     [수집완료]
1213 페이지 수집중입니다.... ing     [수집완료]
1214 페이지 수집중입니다.... ing     [수집완료]
1215 페이지 수집중입니다.... ing     [수집완료]
1216 페이지 수집중입니다.... ing     [수집완료]
1217 페이지 수집중입니다.... ing     [수집완료]
1218 페이지 수집중입니다.... ing     [수집완료]
1219 페이지 수집중입니다.... ing     [수집완료]
1220 페이지 수집중입니다.... ing     [수집완료]
1221 페이지 수집중입니다.... ing     [수집완료]
1222 페이지 수집중입니다.... ing     [수집완료]
1223 페이지 수집중입니다.... ing     [수집완료]
1224 페이지 수집중입니다.... ing     [수집완료]
1225 페이지 수집중입니다.... ing     [수집완료]
1226 페이지 수집중입니다.... ing     [수집완료]
1227 페이지 수집중입니다.... ing     [수집완료]
1228 페이지 수집중입니다.... ing     [수집완료]
1229 페이지 수집중입니다.... ing     [수집완료]
1230 페이지 수집중입니다.... ing     [수집완료]
1231 페이지 수집중입니다.... ing     [수집완료]
1232 페이지 수집중입니다.... ing     [수집완료]
1233 페이지 수집중입니다.... ing     [수집완료]
1234 페이지 수집중입니다.... 

1441 페이지 수집중입니다.... ing     [수집완료]
1442 페이지 수집중입니다.... ing     [수집완료]
1443 페이지 수집중입니다.... ing     [수집완료]
1444 페이지 수집중입니다.... ing     [수집완료]
1445 페이지 수집중입니다.... ing     [수집완료]
1446 페이지 수집중입니다.... ing     [수집완료]
1447 페이지 수집중입니다.... ing     [수집완료]
1448 페이지 수집중입니다.... ing     [수집완료]
1449 페이지 수집중입니다.... ing     [수집완료]
1450 페이지 수집중입니다.... ing     [수집완료]
1451 페이지 수집중입니다.... ing     [수집완료]
1452 페이지 수집중입니다.... ing     [수집완료]
1453 페이지 수집중입니다.... ing     [수집완료]
1454 페이지 수집중입니다.... ing     [수집완료]
1455 페이지 수집중입니다.... ing     [수집완료]
1456 페이지 수집중입니다.... ing     [수집완료]
1457 페이지 수집중입니다.... ing     [수집완료]
1458 페이지 수집중입니다.... ing     [수집완료]
1459 페이지 수집중입니다.... ing     [수집완료]
1460 페이지 수집중입니다.... ing     [수집완료]
1461 페이지 수집중입니다.... ing     [수집완료]
1462 페이지 수집중입니다.... ing     [수집완료]
1463 페이지 수집중입니다.... ing     [수집완료]
1464 페이지 수집중입니다.... ing     [수집완료]
1465 페이지 수집중입니다.... ing     [수집완료]
1466 페이지 수집중입니다.... ing     [수집완료]
1467 페이지 수집중입니다.... ing     [수집완료]
1468 페이지 수집중입니다.... ing     [수집완료]
1469 페이지 수집중입니다.... 

In [23]:
data = BeautifulSoup(driver.page_source, "lxml")

In [24]:
data1[0].find_all("div", "bl_date light")[0].text[-8:]

'17.10.31'

In [25]:
title

['직장 상사 갑질로 인해 죽고 싶어요',
 '택시생존권을 보장해주세요.카카오 카풀영업을 방관하지마세요.',
 '악성 고객으로 부터 저희를 지킬 수 있게 도와주세요',
 '고졸 특채  제고해주세요',
 '코이카 해외자문관의 문제점',
 '공공기관 장애인 근로자 평가후 무기계약직 전환 시켜 달라.',
 '공공기관 (공단 공사등)채용 비리 잡아주시죠',
 '수출입은행의 썩어문드러진 운영! 왜 방치하는가?',
 '일자리',
 "죽고   싶은   대한민국,  자살   공화국    대한민국,   인권위는   필요없다!    '국가   생존권   위원회'를    신설하라!",
 '서희건설',
 '알바  시급  하고  월급  을   제대로 주는   대기업하고  중소기업   상인들 알바 주는 거 신고 합니다',
 '경제난과 실업난을 동시에 해결하자',
 '채용 비리 근절해주세요',
 '서욹교통공사 정규직  전환 비리 강원랜드보다 죄질이 더하다.',
 '꼭한번 봐주세요',
 '자폐할당제를 마련해주세요',
 '공공기관 비정규직 정규직전환 퇴출시켜주세요',
 '65세에서70세까지 장년근로자에게 고용보험과 실업급여 해택요망',
 '공직자 퇴직후 공공기관 및 대기업 재취업을 막아주세요',
 '정년퇴직자 공공기관, 공공단체 임원으로의  임명 제한을 통한 일자리 증진',
 '최저임금 그만 올려라 나라 망한다',
 '우리 자녀들, 친구들, 지인들 모두 임금체불없는 회사, 건강한 사회에서 일할 수 있길 바랍니다.',
 '공평한 기회와 성공의 가능성(연구사)',
 '공무원 인터넷 강의 폭리, 강의 하나가 50만원 , 교재는 5만원 참여인원',
 '9급도 한국사 시험을 한국사능력검정시험으로 대체해야 합니다.',
 '반려동물 2천만시대, 수의대 입학정원 늘려야',
 '연 1만명 규모로   방송대 로스쿨 설치를 청원합니다.',
 '약대 입학정원을 연 5천명 수준으로',
 '세계를 선도할 의사 약사 치과의사 100만명 양성해야',
 '한달에 1억 벌기는 문제도 아니라는 수의사들',
 '평균연봉 82

In [26]:
url = "https://www1.president.go.kr/petitions/category?c=36&only=2&page=1&order=1"
html = urlopen(url)
soup = BeautifulSoup(html, "lxml")

In [27]:
soup.find_all("a")

[<a href="#cont_view"><strong>본문으로 바로가기</strong></a>,
 <a href="#sitemap"><strong>하단메뉴 바로가기</strong></a>,
 <a href="/" title="메인페이지로 이동"><img alt="청와대 로고" src="http://www.president.go.kr/cwdimg/main/logo.png" style="width:100%;"/></a>,
 <a href="/calendar">문재인 대통령</a>,
 <a href="/calendar">공개일정</a>,
 <a href="/president-journey">걸어온길</a>,
 <a href="/c/president-speeches">대통령의 말과 글</a>,
 <a href="/c/first-lady-quotations">김정숙의 말과 글</a>,
 <a href="/live-speeches">청와대 뉴스룸</a>,
 <a href="/live-speeches">청와대 온에어</a>,
 <a href="/c/briefings">청와대 브리핑</a>,
 <a href="/c/oversea-journals">해외언론보도</a>,
 <a href="/c/blue-house-stories">청와대 이야기</a>,
 <a href="/TodayShot">오늘의 한 장</a>,
 <a href="/econo_num">숫자로 읽는 우리 경제</a>,
 <a href="/history">청와대 일기</a>,
 <a href="/policies">정책자료</a>,
 <a href="/policies">정책현안</a>,
 <a href="http://www.korea.kr/policy/actuallyList.do?pWise=main&amp;pWiseMain=A25" rel="noopener noreferrer" target="_blank" title="정책브리핑 새창열림">정책브리핑</a>,
 <a href="/government-projects">

In [28]:
len(date)

15820

In [29]:
len(count)

15820

In [30]:
title[0]

'직장 상사 갑질로 인해 죽고 싶어요'

In [31]:
date[0]

'18.11.15'

In [32]:
count[0]

'8명'

In [33]:
title[-1]

'최저임금 인상이 가져올 파장'

In [34]:
date[-1]

'18.01.01'

In [35]:
count[-1]

'0명'

### DataFrame 만들기 & 파일로 저장하기

In [36]:
df_temp = {'Title' : title, 'Date' : date, 'count' : count}
df = pd.DataFrame(df_temp)

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15820 entries, 0 to 15819
Data columns (total 3 columns):
Title    15820 non-null object
Date     15820 non-null object
count    15820 non-null object
dtypes: object(3)
memory usage: 370.9+ KB


In [38]:
df.head()

,Title,Date,count
0,직장 상사 갑질로 인해 죽고 싶어요,18.11.15,8명
1,택시생존권을 보장해주세요.카카오 카풀영업을 방관하지마세요.,18.11.15,39명
2,악성 고객으로 부터 저희를 지킬 수 있게 도와주세요,18.11.15,16명
3,고졸 특채 제고해주세요,18.11.15,4명
4,코이카 해외자문관의 문제점,18.11.15,0명


In [39]:
df.tail()

,Title,Date,count
15815,계약직 임산부 야근 및 부당 처우,18.01.01,0명
15816,계약직 임산부 야근 및 부당 처우,18.01.01,0명
15817,"아무리 일해도 가난한, 나는 고졸입니다",18.01.01,2명
15818,고졸 일자리 정책 너무 없습니다.,18.01.01,1명
15819,최저임금 인상이 가져올 파장,18.01.01,0명


In [40]:
df.to_csv('teample_1_result/kookmin/president_all(%s - %s).csv' % (end_date, start_date), sep=',', encoding='UTF-8')
print('TeamMission/president_all(%s - %s).csv 파일을 생성했습니다.' % (end_date, start_date))      

TeamMission/president_all(18.01.01 - 18.11.15).csv 파일을 생성했습니다.
